In [38]:
"""
Interactive Sankey Diagram for Delivery Challenges Analysis
Author: Xiaosu Li
Purpose: Visualize relationships between challenges, solutions, and implementation examples
"""

import plotly.graph_objects as go
import numpy as np
import os
import json

# Fix for numpy compatibility issue
try:
    if not hasattr(np, 'bool8'):
        np.bool8 = np.bool_
except AttributeError:
    pass

print("✓ Dependencies loaded successfully")

✓ Dependencies loaded successfully


In [50]:
def create_adaptive_node_sankey(invert_order=True):
    """Create a Sankey diagram where node sizes adapt to actual stream flows"""
    
    # Color palette - Controls opacity of nodes and links
    morandi_colors = {
        'blue_green': {'primary': 'rgba(106, 144, 138, 0.5)', 'secondary': 'rgba(106, 144, 138, 0.3)'},
        'purple': {'primary': 'rgba(141, 132, 158, 0.5)', 'secondary': 'rgba(141, 132, 158, 0.3)'},
        'orange': {'primary': 'rgba(198, 160, 123, 0.5)', 'secondary': 'rgba(198, 160, 123, 0.3)'},
        'red': {'primary': 'rgba(186, 134, 120, 0.5)', 'secondary': 'rgba(186, 134, 120, 0.3)'},
        'neutral': {'primary': 'rgba(164, 157, 151, 0.5)', 'secondary': 'rgba(164, 157, 151, 0.3)'}
    }
    
    # Node labels - 19 nodes total: 4 challenges + 5 solutions + 10 examples
    nodes = [
        # Layer 1: Challenges (indices 0-3)
        "C1: Unfavorable Cost Structure", "C2: Logistical & Network Inefficiency", 
        "C3: Infrastructure Deficiency", "C4: Emission Concerns",
        
        # Layer 2: Solutions (indices 4-8)
        "S1: Collaborative Networks", "S2: Alternative Delivery Points",
        "S3: Route & Dispatch Optimization", "S4: Dynamic Workforce", "S5: Integrated Demand Management",
        
        # Layer 3: Examples (indices 9-18)
        "Horizontal", "Vertical", "Fixed Micro-Depots & Hubs", "Mobile Infrastructure", "Community-based Drop-Off",
        "Dynamic Dispatching", "Dynamic Routing", "Delivery Helpers", "Crowdshipping", "Dynamic Pricing"
    ]
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # FLOW VALUES - THIS IS WHERE YOU CHANGE THE STREAM THICKNESS
    # Format: (source_index, target_index, flow_value)
    # Higher flow_value = thicker stream
    # ═══════════════════════════════════════════════════════════════════════════════════
    flows = [
        # ─── LAYER 1 → LAYER 2: Challenges to Solutions ───
        # C1 (index 0) flows:
        (0, 4, 0.5),    # C1 → S1 (Collaborative Networks) - CHANGE THIS VALUE
        (0, 6, 0.5),  # C1 → S3 (Route Optimization) - CHANGE THIS VALUE
        (0, 8, 0.5),    # C1 → S5 (Demand Management) - CHANGE THIS VALUE
        
        # C2 (index 1) flows:
        (1, 4, 0.5),  # C2 → S1 (Collaborative Networks) - CHANGE THIS VALUE
        (1, 5, 0.5),    # C2 → S2 (Alternative Delivery) - CHANGE THIS VALUE
        (1, 6, 0.5),    # C2 → S3 (Route Optimization) - CHANGE THIS VALUE
        (1, 8, 0.5),  # C2 → S5 (Demand Management) - CHANGE THIS VALUE
        
        # C3 (index 2) flows:
        (2, 5, 0.5),    # C3 → S2 (Alternative Delivery) - CHANGE THIS VALUE
        (2, 7, 1),    # C3 → S4 (Dynamic Workforce) - CHANGE THIS VALUE
        
        # C4 (index 3) flows:
        (3, 4, 0.5),  # C4 → S1 (Collaborative Networks) - CHANGE THIS VALUE
        (3, 6, 0.5),    # C4 → S3 (Route Optimization) - CHANGE THIS VALUE
        
        # ─── LAYER 2 → LAYER 3: Solutions to Examples ───
        # S1 (index 4) flows:
        (4, 9, 0.5),    # S1 → Horizontal - CHANGE THIS VALUE
        (4, 10, 0.5),   # S1 → Vertical - CHANGE THIS VALUE
        (4, 13, 0.5),   # S1 → Community Drop-Off - CHANGE THIS VALUE
        
        # S2 (index 5) flows:
        (5, 11, 0.33),     # S2 → Micro-Depots - CHANGE THIS VALUE
        (5, 12, 0.33),   # S2 → Mobile Infrastructure - CHANGE THIS VALUE
        (5, 13, 0.33),   # S2 → Community Drop-Off - CHANGE THIS VALUE

        
        # S3 (index 6) flows:
        (6, 14, 0.75),   # S3 → Dynamic Dispatching - CHANGE THIS VALUE
        (6, 15, 0.75),   # S3 → Dynamic Routing - CHANGE THIS VALUE
        
        # S4 (index 7) flows:
        (7, 16, 0.33),   # S4 → Delivery Helpers - CHANGE THIS VALUE
        (7, 17, 0.33),   # S4 → Crowdshipping - CHANGE THIS VALUE
        (7, 13, 0.33),   # S4 → Community Drop-Off - CHANGE THIS VALUE
        
        # S5 (index 8) flows:
        (8, 18, 1)    # S5 → Dynamic Pricing - CHANGE THIS VALUE
    ]

    # Node color assignment
    challenge_colors = ['purple', 'red', 'orange', 'blue_green']
    node_colors = []
    
    # Challenge nodes (Layer 1) - use primary colors
    for i in range(4):
        node_colors.append(morandi_colors[challenge_colors[i]]['primary'])
    
    # Solution nodes (Layer 2) - use secondary colors
    concept_mapping = {
        4: 'purple', 5: 'red', 6: 'blue_green', 7: 'orange', 8: 'neutral'
    }
    for i in range(4, 9):
        node_colors.append(morandi_colors[concept_mapping[i]]['secondary'])
    
    # Example nodes (Layer 3) - use secondary colors
    example_mapping = {
        9: 'purple', 10: 'purple', 11: 'red', 12: 'red', 13: 'red',
        14: 'blue_green', 15: 'blue_green', 16: 'orange', 17: 'orange', 18: 'neutral'
    }
    for i in range(9, 19):
        node_colors.append(morandi_colors[example_mapping[i]]['secondary'])
    
    # Link colors - inherit from source nodes
    link_colors = []
    for source, target, value in flows:
        if source < 4:  # Links from challenges
            color_key = challenge_colors[source]
            link_colors.append(morandi_colors[color_key]['primary'])
        else:  # Links from solutions
            link_colors.append(node_colors[source])
    
    # Node positioning (vertical layout)
    if invert_order:
        challenges_y = [0.2, 0.4, 0.6, 0.8]
        solutions_y = [0.15, 0.325, 0.5, 0.675, 0.85]
        examples_y = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
    else:
        challenges_y = [0.8, 0.6, 0.4, 0.2]
        solutions_y = [0.85, 0.675, 0.5, 0.325, 0.15]
        examples_y = [0.95, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    
    y_positions = challenges_y + solutions_y + examples_y
    
    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,                                          # Space between nodes
            thickness=8,                                     # Node thickness (height)
            line=dict(color="rgba(100,100,100,0.3)", width=0.5),  # Node border
            label=nodes,
            color=node_colors,
            x=[0.1]*4 + [0.5]*5 + [0.85]*10,                # Horizontal positions
            y=y_positions                                     # Vertical positions
        ),
        link=dict(
            source=[flow[0] for flow in flows],               # Source node indices
            target=[flow[1] for flow in flows],               # Target node indices
            value=[flow[2] for flow in flows],                # Flow values (stream thickness)
            color=link_colors,
            hovertemplate='%{source.label}<br>→<br>%{target.label}<br>Flow: %{value}<extra></extra>'
        )
    )])
    
    # Layout configuration
    fig.update_layout(
        title=dict(
            text="",
            font=dict(size=17, color='rgb(60, 60, 60)'),
            x=0.5, xanchor='center'
        ),
        font=dict(size=17, color='rgb(60, 60, 60)'),
        width=1280, height=720,
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor='rgba(250, 250, 250, 1)',
        plot_bgcolor='rgba(250, 250, 250, 1)'
    )
    
    return fig


# ═══════════════════════════════════════════════════════════════════════════════════
# INSTRUCTIONS FOR MODIFYING FLOW VALUES:
# ═══════════════════════════════════════════════════════════════════════════════════
print("📊 HOW TO CHANGE FLOW VALUES")
print("=" * 50)
print("1. Find the 'flows = [' section above")
print("2. Each flow is: (source, target, value)")
print("3. Change the THIRD number to adjust stream thickness")
print("4. Example: (0, 4, 2) → (0, 4, 5) makes stream thicker")
print("5. Recommended range: 0.5 to 5.0 for good visual balance")
print("")
print("💡 FLOW VALUE GUIDELINES:")
print("• 0.5-1.0: Thin streams")
print("• 1.5-2.5: Medium streams") 
print("• 3.0-5.0: Thick streams")

# Create and show adaptive version
adaptive_fig = create_adaptive_node_sankey(invert_order=True)
adaptive_fig.show()

print("\n✓ Adaptive node sizing version created!")
print("• Optimized node thickness: 8 pixels")
print("• Varied flow values: 0.5-3.0 for better proportions")
print("• Nodes better match their stream volumes")

📊 HOW TO CHANGE FLOW VALUES
1. Find the 'flows = [' section above
2. Each flow is: (source, target, value)
3. Change the THIRD number to adjust stream thickness
4. Example: (0, 4, 2) → (0, 4, 5) makes stream thicker
5. Recommended range: 0.5 to 5.0 for good visual balance

💡 FLOW VALUE GUIDELINES:
• 0.5-1.0: Thin streams
• 1.5-2.5: Medium streams
• 3.0-5.0: Thick streams



✓ Adaptive node sizing version created!
• Optimized node thickness: 8 pixels
• Varied flow values: 0.5-3.0 for better proportions
• Nodes better match their stream volumes


In [52]:
def export_sankey_for_ppt(fig, output_path, filename="sankey_diagram"):
    """
    Export Sankey diagram in multiple formats for PowerPoint integration
    
    Parameters:
    fig: plotly figure object
    output_path: str, directory path to save files
    filename: str, base filename (without extension)
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    export_files = {}
    
    # 1. Interactive HTML (CDN version)
    html_file = os.path.join(output_path, f"{filename}.html")
    fig.write_html(html_file, 
                   include_plotlyjs='cdn',
                   config={'displayModeBar': False})
    export_files['html'] = html_file
    print(f"Interactive HTML saved: {html_file}")
    
    # 2. Standalone HTML (no internet required)
    html_standalone = os.path.join(output_path, f"{filename}_standalone.html")
    fig.write_html(html_standalone, 
                   include_plotlyjs=True,
                   config={'displayModeBar': False})
    export_files['html_standalone'] = html_standalone
    print(f"Standalone HTML saved: {html_standalone}")
    
    # 3. Static PNG (16:9 ratio)
    png_file = os.path.join(output_path, f"{filename}.png")
    try:
        fig.write_image(png_file, width=1920, height=1080, scale=2)
        export_files['png'] = png_file
        print(f"PNG image saved: {png_file}")
    except Exception as e:
        print(f"PNG export failed (install kaleido: pip install kaleido): {e}")
    
    # 4. SVG vector format
    svg_file = os.path.join(output_path, f"{filename}.svg")
    try:
        fig.write_image(svg_file, width=1920, height=1080)
        export_files['svg'] = svg_file
        print(f"SVG image saved: {svg_file}")
    except Exception as e:
        print(f"SVG export failed: {e}")
    
    # 5. PowerPoint-optimized HTML
    ppt_html_file = os.path.join(output_path, f"{filename}_ppt_optimized.html")
    create_ppt_optimized_html(fig, ppt_html_file)
    export_files['ppt_html'] = ppt_html_file
    print(f"PPT-optimized HTML saved: {ppt_html_file}")
    
    return export_files


def create_ppt_optimized_html(fig, output_path):
    """Create PowerPoint-optimized HTML with custom styling for 16:9 aspect ratio"""
    
    # Configure figure for PowerPoint
    fig_copy = fig
    fig_copy.update_layout(
        width=1600, height=900,
        margin=dict(l=30, r=30, t=50, b=30),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(size=16, color='black')
    )
    
    # HTML template for PowerPoint
    html_template = """<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Sankey Diagram for PowerPoint</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{ margin: 0; padding: 10px; font-family: Arial, sans-serif; background-color: white; }}
        #plotly-div {{ width: 100%; height: 90vh; }}
    </style>
</head>
<body>
    <div id="plotly-div"></div>
    <script>
        var plotData = {plot_data};
        var layout = {layout};
        var config = {{ displayModeBar: false, responsive: true, displaylogo: false }};
        
        Plotly.newPlot('plotly-div', plotData, layout, config);
        window.addEventListener('resize', function() {{ Plotly.Plots.resize('plotly-div'); }});
    </script>
</body>
</html>"""
    
    # Generate HTML content
    plot_json = fig_copy.to_json()
    plot_dict = json.loads(plot_json)
    
    html_content = html_template.format(
        plot_data=json.dumps(plot_dict['data']),
        layout=json.dumps(plot_dict['layout'])
    )
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)


def export_example():
    """Complete example of creating and exporting Sankey diagram"""
    
    # Create the diagram
    fig = create_adaptive_node_sankey()
    
    # Define output path (current working directory for illustrations)
    output_directory = "/Users/xiaosu/Nextcloud/00 CSA Shared Data/Promotion Xiaosu Li Shared Ressource/0-Current-Work/Illustrations"
    
    # Export in multiple formats
    exported_files = export_sankey_for_ppt(
        fig=fig,
        output_path=output_directory,
        filename="delivery_challenges_sankey"
    )
    
    # Summary
    print("\n=== EXPORT COMPLETE ===")
    print("Files created:")
    for format_type, filepath in exported_files.items():
        print(f"  {format_type}: {filepath}")
    
    print("\n=== POWERPOINT INTEGRATION METHODS ===")
    print("1. WEB OBJECT: Insert > Get Add-ins > Web Viewer")
    print("2. HYPERLINK: Insert > Link > Link to File") 
    print("3. STATIC IMAGE: Insert > Pictures > From File")
    
    return exported_files


# Quick export function
def quick_export(output_path="./sankey_export", filename="delivery_challenges_sankey"):
    """Quick export function - just specify path and filename"""
    fig = create_adaptive_node_sankey()
    return export_sankey_for_ppt(fig, output_path, filename)


# Execute export example
print("Export functions ready. Run export_example() to create files.")
if __name__ == "__main__":
    exported_files = export_example()
    print("Exported files:", exported_files)    

Export functions ready. Run export_example() to create files.
Interactive HTML saved: /Users/xiaosu/Nextcloud/00 CSA Shared Data/Promotion Xiaosu Li Shared Ressource/0-Current-Work/Illustrations/delivery_challenges_sankey.html
Standalone HTML saved: /Users/xiaosu/Nextcloud/00 CSA Shared Data/Promotion Xiaosu Li Shared Ressource/0-Current-Work/Illustrations/delivery_challenges_sankey_standalone.html
PNG export failed (install kaleido: pip install kaleido): 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

SVG export failed: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

PPT-optimized HTML saved: /Users/xiaosu/Nextcloud/00 CSA Shared Data/Promotion Xiaosu Li Shared Ressource/0-Current-Work/Illustrations/delivery_challenges_sankey_ppt_optimized.html

=== EXPORT COMPLETE ===
Files created:
  html: /Users/xiaosu/Nextclou

In [ ]:
def create_delivery_challenges_sankey(invert_order=True):
    """
    Create an interactive Sankey diagram showing the flow from delivery challenges 
    to conceptual solutions to specific examples using Morandi color tones.
    
    Parameters:
    invert_order (bool): If True, displays C4 at top and C1 at bottom. If False, C1 at top and C4 at bottom.
    """
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # COLOR PALETTE - Change opacity values here (0.0 = transparent, 1.0 = opaque)
    # ═══════════════════════════════════════════════════════════════════════════════════
    morandi_colors = {
        'blue_green': {
            'primary': 'rgba(106, 144, 138, 1.0)',      # Challenge layer opacity
            'secondary': 'rgba(106, 144, 138, 0.4)'     # Solution/Example layer opacity
        },
        'purple': {
            'primary': 'rgba(141, 132, 158, 1.0)',      # Challenge layer opacity
            'secondary': 'rgba(141, 132, 158, 0.4)'     # Solution/Example layer opacity
        },
        'orange': {
            'primary': 'rgba(198, 160, 123, 1.0)',      # Challenge layer opacity
            'secondary': 'rgba(198, 160, 123, 0.4)'     # Solution/Example layer opacity
        },
        'red': {
            'primary': 'rgba(186, 134, 120, 1.0)',      # Challenge layer opacity
            'secondary': 'rgba(186, 134, 120, 0.4)'     # Solution/Example layer opacity
        },
        'neutral': {
            'primary': 'rgba(164, 157, 151, 1.0)',      # Challenge layer opacity
            'secondary': 'rgba(164, 157, 151, 0.4)'     # Solution/Example layer opacity
        }
    }
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # NODE LABELS - 19 nodes total organized in 3 layers
    # ═══════════════════════════════════════════════════════════════════════════════════
    nodes = [
        # ─── LAYER 1: Challenges (indices 0-3) ───
        "C1: Unfavorable Cost Structure",           # Index 0
        "C2: Logistical & Network Inefficiency",    # Index 1
        "C3: Infrastructure Deficiency",            # Index 2
        "C4: Environmental Pressures",              # Index 3
        
        # ─── LAYER 2: Solutions (indices 4-8) ───
        "S1: Collaborative Networks",                        # Index 4
        "S2: Alternative Delivery Points/Infrastructure",    # Index 5
        "S3: Route & Dispatch Optimization",                 # Index 6
        "S4: Dynamic Workforce",                             # Index 7
        "S5: Integrated Demand Management",                  # Index 8
        
        # ─── LAYER 3: Examples (indices 9-18) ───
        "Horizontal", "Vertical",                            # Indices 9, 10 (from S1)
        "Micro-Depots & Hubs", "Mobile Infrastructure", "Community-based Drop-Off",  # Indices 11, 12, 13 (from S2)
        "Dynamic Dispatching", "Dynamic Routing",           # Indices 14, 15 (from S3)
        "Delivery Helpers", "Crowdsourcing",                # Indices 16, 17 (from S4)
        "Dynamic Pricing"                                    # Index 18 (from S5)
    ]
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # *** FLOW VALUES - THIS IS WHERE YOU CHANGE STREAM THICKNESS ***
    # Format: (source_index, target_index, flow_value)
    # Higher flow_value = thicker stream in the diagram
    # ═══════════════════════════════════════════════════════════════════════════════════
    flows = [
        # ─────────────────────────────────────────────────────────────────────────────────
        # LAYER 1 → LAYER 2: Challenges to Solutions
        # ─────────────────────────────────────────────────────────────────────────────────
        
        # C1 (Unfavorable Cost Structure) outflows:
        (0, 4, 25),  # C1 → S1 (Collaborative Networks) - CHANGE VALUE HERE
        (0, 6, 20),  # C1 → S3 (Route Optimization) - CHANGE VALUE HERE
        (0, 8, 15),  # C1 → S5 (Demand Management) - CHANGE VALUE HERE
        
        # C2 (Logistical & Network Inefficiency) outflows:
        (1, 4, 30),  # C2 → S1 (Collaborative Networks) - CHANGE VALUE HERE
        (1, 5, 35),  # C2 → S2 (Alternative Delivery Points) - CHANGE VALUE HERE
        (1, 6, 25),  # C2 → S3 (Route Optimization) - CHANGE VALUE HERE
        (1, 8, 20),  # C2 → S5 (Demand Management) - CHANGE VALUE HERE
        
        # C3 (Infrastructure Deficiency) outflows:
        (2, 5, 40),  # C3 → S2 (Alternative Delivery Points) - CHANGE VALUE HERE
        (2, 7, 30),  # C3 → S4 (Dynamic Workforce) - CHANGE VALUE HERE
        
        # C4 (Environmental Pressures) outflows:
        (3, 4, 20),  # C4 → S1 (Collaborative Networks) - CHANGE VALUE HERE
        (3, 6, 30),  # C4 → S3 (Route Optimization) - CHANGE VALUE HERE
        
        # ─────────────────────────────────────────────────────────────────────────────────
        # LAYER 2 → LAYER 3: Solutions to Examples
        # ─────────────────────────────────────────────────────────────────────────────────
        
        # S1 (Collaborative Networks) outflows:
        (4, 9, 30),   # S1 → Horizontal Collaboration - CHANGE VALUE HERE
        (4, 10, 45),  # S1 → Vertical Collaboration - CHANGE VALUE HERE
        (4, 13, 15),  # S1 → Community Drop-Off - CHANGE VALUE HERE
        
        # S2 (Alternative Delivery Points) outflows:
        (5, 11, 25),  # S2 → Micro-Depots & Hubs - CHANGE VALUE HERE
        (5, 12, 30),  # S2 → Mobile Infrastructure - CHANGE VALUE HERE
        (5, 13, 20),  # S2 → Community Drop-Off - CHANGE VALUE HERE
        (5, 13, 10),  # S2 → Community Drop-Off (additional flow) - CHANGE VALUE HERE
        
        # S3 (Route & Dispatch Optimization) outflows:
        (6, 14, 35),  # S3 → Dynamic Dispatching - CHANGE VALUE HERE
        (6, 15, 20),  # S3 → Dynamic Routing - CHANGE VALUE HERE
        
        # S4 (Dynamic Workforce) outflows:
        (7, 16, 25),  # S4 → Delivery Helpers - CHANGE VALUE HERE
        (7, 17, 25),  # S4 → Crowdsourcing - CHANGE VALUE HERE
        (7, 13, 12),  # S4 → Community Drop-Off - CHANGE VALUE HERE
        
        # S5 (Integrated Demand Management) outflows:
        (8, 18, 35),  # S5 → Dynamic Pricing - CHANGE VALUE HERE
    ]
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # COLOR ASSIGNMENT FOR NODES
    # ═══════════════════════════════════════════════════════════════════════════════════
    challenge_colors = ['purple', 'red', 'orange', 'blue_green']
    node_colors = []
    
    # Challenge nodes (Layer 1) - use primary colors (full opacity)
    for i in range(4):
        node_colors.append(morandi_colors[challenge_colors[i]]['primary'])
    
    # Solution nodes (Layer 2) - use secondary colors (lower opacity)
    concept_mapping = [
        morandi_colors['purple']['secondary'],     # S1 matches C1 (purple)
        morandi_colors['red']['secondary'],        # S2 matches C2 (red)
        morandi_colors['orange']['secondary'],     # S3 matches C3 (orange)
        morandi_colors['blue_green']['secondary'], # S4 matches C4 (blue_green)
        morandi_colors['neutral']['secondary']     # S5 neutral
    ]
    node_colors.extend(concept_mapping)
    
    # Example nodes (Layer 3) - use secondary colors matching parent solutions
    example_mapping = [
        morandi_colors['purple']['secondary'],     # Horizontal (from S1)
        morandi_colors['purple']['secondary'],     # Vertical (from S1)
        morandi_colors['red']['secondary'],        # Micro-Depots (from S2)
        morandi_colors['red']['secondary'],        # Mobile Infrastructure (from S2)
        morandi_colors['red']['secondary'],        # Community Drop-Off (from S2)
        morandi_colors['orange']['secondary'],     # Dynamic Dispatching (from S3)
        morandi_colors['orange']['secondary'],     # Dynamic Routing (from S3)
        morandi_colors['blue_green']['secondary'], # Delivery Helpers (from S4)
        morandi_colors['blue_green']['secondary'], # Crowdsourcing (from S4)
        morandi_colors['neutral']['secondary']     # Dynamic Pricing (from S5)
    ]
    node_colors.extend(example_mapping)
    
    # Link colors - inherit from source nodes with reduced opacity
    link_colors = []
    for source, target, value in flows:
        if source < 4:  # Links from challenges
            color_key = challenge_colors[source]
            link_colors.append(morandi_colors[color_key]['secondary'])
        else:  # Links from solutions - make even more transparent
            link_colors.append(node_colors[source].replace('0.4', '0.2'))
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # NODE POSITIONING (VERTICAL LAYOUT)
    # ═══════════════════════════════════════════════════════════════════════════════════
    if invert_order:
        challenges_y = [0.2, 0.4, 0.6, 0.8]
        solutions_y = [0.15, 0.325, 0.5, 0.675, 0.85]
        examples_y = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
    else:
        challenges_y = [0.8, 0.6, 0.4, 0.2]
        solutions_y = [0.85, 0.675, 0.5, 0.325, 0.15]
        examples_y = [0.95, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    
    y_positions = challenges_y + solutions_y + examples_y
    
    # ═══════════════════════════════════════════════════════════════════════════════════
    # CREATE SANKEY DIAGRAM
    # ═══════════════════════════════════════════════════════════════════════════════════
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=10,                                              # Space between nodes
            thickness=15,                                        # Node thickness (height)
            line=dict(color="rgba(100,100,100,0.5)", width=1),  # Node border
            label=nodes,
            color=node_colors,
            x=[0.1]*4 + [0.5]*5 + [0.85]*10,                    # Horizontal positions
            y=y_positions                                         # Vertical positions
        ),
        link=dict(
            source=[flow[0] for flow in flows],                  # Source node indices
            target=[flow[1] for flow in flows],                  # Target node indices
            value=[flow[2] for flow in flows],                   # Flow values (stream thickness)
            color=link_colors,
            hovertemplate='%{source.label} → %{target.label}<br>Flow: %{value}<extra></extra>'
        )
    )])
    
    # Layout configuration
    fig.update_layout(
        font=dict(size=15, color='rgb(60, 60, 60)'),
        width=1280,
        height=720,
        margin=dict(l=20, r=20, t=70, b=20),
        paper_bgcolor='rgba(248, 248, 248, 0.9)',
        plot_bgcolor='rgba(248, 248, 248, 0.9)'
    )
    
    return fig


def main(invert_order=True):
    """Main function to create and show the Sankey diagram"""
    fig = create_delivery_challenges_sankey(invert_order=invert_order)
    
    print("Interactive Sankey Diagram Features:")
    print("• Hover over nodes and links for detailed information")
    print("• Click and drag to pan the diagram")
    print("• Use scroll wheel to zoom in/out")
    print("• Links show flow direction and magnitude")
    
    fig.show()
    return fig


# ═══════════════════════════════════════════════════════════════════════════════════
# *** INSTRUCTIONS FOR MODIFYING FLOW VALUES ***
# ═══════════════════════════════════════════════════════════════════════════════════
print("📊 FLOW VALUE MODIFICATION GUIDE")
print("=" * 60)
print("🎯 WHERE TO CHANGE: Look for 'flows = [' section above")
print("")
print("📋 FLOW FORMAT: (source_index, target_index, flow_value)")
print("   • source_index: Starting node (0-18)")
print("   • target_index: Ending node (0-18)")  
print("   • flow_value: Stream thickness (CHANGE THIS NUMBER)")
print("")
print("💡 VALUE RECOMMENDATIONS:")
print("   • Small flows: 5-15")
print("   • Medium flows: 20-35")
print("   • Large flows: 40-50")
print("")
print("🔧 EXAMPLES:")
print("   • (0, 4, 25) → (0, 4, 40)  makes C1→S1 stream thicker")
print("   • (1, 5, 35) → (1, 5, 10)  makes C2→S2 stream thinner")

# Execute and create the diagram
if __name__ == "__main__":
    sankey_fig = main()
    print("✓ Main Sankey diagram created successfully!")

Interactive Sankey Diagram Features:
• Hover over nodes and links for detailed information
• Click and drag to pan the diagram
• Use scroll wheel to zoom in/out
• Links show flow direction and magnitude


✓ Main Sankey diagram created successfully!


In [37]:
def export_sankey_for_ppt(fig, output_path, filename="sankey_diagram"):
    """
    Export Sankey diagram in multiple formats for PowerPoint integration
    
    Parameters:
    fig: plotly figure object
    output_path: str, directory path to save files
    filename: str, base filename (without extension)
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    export_files = {}
    
    # 1. Interactive HTML (CDN version)
    html_file = os.path.join(output_path, f"{filename}.html")
    fig.write_html(html_file, 
                   include_plotlyjs='cdn',
                   config={'displayModeBar': False})
    export_files['html'] = html_file
    print(f"Interactive HTML saved: {html_file}")
    
    # 2. Standalone HTML (no internet required)
    html_standalone = os.path.join(output_path, f"{filename}_standalone.html")
    fig.write_html(html_standalone, 
                   include_plotlyjs=True,
                   config={'displayModeBar': False})
    export_files['html_standalone'] = html_standalone
    print(f"Standalone HTML saved: {html_standalone}")
    
    # 3. Static PNG (16:9 ratio)
    png_file = os.path.join(output_path, f"{filename}.png")
    try:
        fig.write_image(png_file, width=1920, height=1080, scale=2)
        export_files['png'] = png_file
        print(f"PNG image saved: {png_file}")
    except Exception as e:
        print(f"PNG export failed (install kaleido: pip install kaleido): {e}")
    
    # 4. SVG vector format
    svg_file = os.path.join(output_path, f"{filename}.svg")
    try:
        fig.write_image(svg_file, width=1920, height=1080)
        export_files['svg'] = svg_file
        print(f"SVG image saved: {svg_file}")
    except Exception as e:
        print(f"SVG export failed: {e}")
    
    # 5. PowerPoint-optimized HTML
    ppt_html_file = os.path.join(output_path, f"{filename}_ppt_optimized.html")
    create_ppt_optimized_html(fig, ppt_html_file)
    export_files['ppt_html'] = ppt_html_file
    print(f"PPT-optimized HTML saved: {ppt_html_file}")
    
    return export_files


def create_ppt_optimized_html(fig, output_path):
    """Create PowerPoint-optimized HTML with custom styling for 16:9 aspect ratio"""
    
    # Configure figure for PowerPoint
    fig_copy = fig
    fig_copy.update_layout(
        width=1600, height=900,
        margin=dict(l=30, r=30, t=50, b=30),
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(size=16, color='black')
    )
    
    # HTML template for PowerPoint
    html_template = """<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Sankey Diagram for PowerPoint</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{ margin: 0; padding: 10px; font-family: Arial, sans-serif; background-color: white; }}
        #plotly-div {{ width: 100%; height: 90vh; }}
    </style>
</head>
<body>
    <div id="plotly-div"></div>
    <script>
        var plotData = {plot_data};
        var layout = {layout};
        var config = {{ displayModeBar: false, responsive: true, displaylogo: false }};
        
        Plotly.newPlot('plotly-div', plotData, layout, config);
        window.addEventListener('resize', function() {{ Plotly.Plots.resize('plotly-div'); }});
    </script>
</body>
</html>"""
    
    # Generate HTML content
    plot_json = fig_copy.to_json()
    plot_dict = json.loads(plot_json)
    
    html_content = html_template.format(
        plot_data=json.dumps(plot_dict['data']),
        layout=json.dumps(plot_dict['layout'])
    )
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)


def export_example():
    """Complete example of creating and exporting Sankey diagram"""
    
    # Create the diagram
    fig = create_delivery_challenges_sankey()
    
    # Define output path (current working directory for illustrations)
    output_directory = "/Users/xiaosu/Nextcloud/00 CSA Shared Data/Promotion Xiaosu Li Shared Ressource/0-Current-Work/Illustrations"
    
    # Export in multiple formats
    exported_files = export_sankey_for_ppt(
        fig=fig,
        output_path=output_directory,
        filename="delivery_challenges_sankey"
    )
    
    # Summary
    print("\n=== EXPORT COMPLETE ===")
    print("Files created:")
    for format_type, filepath in exported_files.items():
        print(f"  {format_type}: {filepath}")
    
    print("\n=== POWERPOINT INTEGRATION METHODS ===")
    print("1. WEB OBJECT: Insert > Get Add-ins > Web Viewer")
    print("2. HYPERLINK: Insert > Link > Link to File") 
    print("3. STATIC IMAGE: Insert > Pictures > From File")
    
    return exported_files


# Quick export function
def quick_export(output_path="./sankey_export", filename="delivery_challenges_sankey"):
    """Quick export function - just specify path and filename"""
    fig = create_delivery_challenges_sankey()
    return export_sankey_for_ppt(fig, output_path, filename)


# Execute export example
print("Export functions ready. Run export_example() to create files.")

Export functions ready. Run export_example() to create files.
